<a href="https://colab.research.google.com/github/ondennis03/AAI2026/blob/main/Exercise_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Exercise 1 Prompt Chaining for a Customer Support AI:
Step 1 (triage) → Step 2 (missing info questions) → Step 3 (resolution plan) → Step 4 (escalation decision)

Iteration evidence: Step 2 includes V1 vs V2 prompt

Tools used: Google Colab + OpenAI API + ChatGPT

In [82]:
!pip -q install openai

In [83]:
import os
from google.colab import userdata

os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")
print("Key loaded:", bool(os.environ.get("OPENAI_API_KEY")))

Key loaded: True


In [84]:
import os
import json
from typing import Dict, Any

from openai import OpenAI

client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))

def chat(prompt: str, model: str = "gpt-4.1-mini") -> str:
    """Send a single prompt and return the assistant's text."""
    resp = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "user", "content": prompt}
        ],
        temperature=0.2,
    )
    return resp.choices[0].message.content.strip()

# Sample customer message

CUSTOMER_MESSAGE = "My order says delivered but I never got it. I need help ASAP."


PROMPT_STEP1 = f"""
You are a customer support triage assistant.

Task: Classify the customer's issue into ONE category:
{{billing, shipping/delivery, returns, technical, account}}.

Output JSON only with keys:
- category (one of the allowed categories)
- urgency (integer 1-5)
- reason (<= 20 words)

Rules:
- Do NOT propose a solution.
- Do NOT ask questions.
- Use only the customer's message.

Customer message: \"\"\"{CUSTOMER_MESSAGE}\"\"\"
"""

step1_raw = chat(PROMPT_STEP1)
print("STEP 1 OUTPUT (raw):")
print(step1_raw)

step1 = json.loads(step1_raw)
print("\nSTEP 1 OUTPUT (parsed):")
print(step1)


PROMPT_STEP2_V1 = f"""
You are a customer support agent.

Use this triage result:
{json.dumps(step1)}

Ask the customer for ONLY the missing info needed to resolve the issue.

Constraints:
- Friendly tone
- Max 3 questions
- No blaming language
- Avoid policy jargon

Output format:
1) One short empathy sentence
2) Numbered questions (1-3)

Return only the formatted message.
"""

step2_v1 = chat(PROMPT_STEP2_V1)
print("\nSTEP 2 OUTPUT (V1):")
print(step2_v1)


PROMPT_STEP2_V2 = f"""
You are a customer support agent.

Context:
- Your job is to collect ONLY the minimum missing details needed to take the next action.
- Do not request info the customer already provided.
- Keep questions specific and easy to answer.

Triage JSON:
{json.dumps(step1)}

Hard constraints:
- Max 3 questions
- Each question must be answerable in one line
- No blaming ("you should have..."), no policy/legal language
- Do NOT propose a solution yet
- Tone: calm, helpful, urgent-appropriate

Output format EXACTLY:
<one empathy sentence>
1) <question>
2) <question>
3) <question if needed; otherwise omit>

Customer message (for reference):
\"\"\"{CUSTOMER_MESSAGE}\"\"\"
"""

step2_v2 = chat(PROMPT_STEP2_V2)
print("\nSTEP 2 OUTPUT (V2 - improved):")
print(step2_v2)


CUSTOMER_ANSWERS = {
    "order_number": "123456",
    "delivery_address": "San Jose, CA 95112",
    "carrier": "UPS",
    "tracking_number": "1Z999AA10123456784",
    "delivered_time": "Today 2:15 PM",
    "delivery_notes_or_photo": "No photo/proof of delivery. No note from driver."
}


PROMPT_STEP3 = f"""
You are a customer support agent.

Goal:
Provide a clear, helpful resolution plan for the customer.

Inputs:
Triage JSON:
{json.dumps(step1)}

Customer answers (JSON):
{json.dumps(CUSTOMER_ANSWERS)}

Constraints:
- Provide a step-by-step plan (4-7 steps)
- Include what YOU will do and what the CUSTOMER can do
- Keep it concise and non-technical
- Do not mention internal policies, legal language, or blame
- End with a single sentence confirming the next update you will provide

Output format:
Title: <short title>
Steps:
1. ...
2. ...
...
Closing: <one sentence>
"""

step3 = chat(PROMPT_STEP3)
print("\nSTEP 3 OUTPUT (Resolution plan):")
print(step3)


PROMPT_STEP4 = f"""
You are a support operations assistant.

Decide whether to escalate this case to a human agent.

Use:
Triage JSON:
{json.dumps(step1)}

Proposed resolution plan:
\"\"\"{step3}\"\"\"

Escalation rules:
- Escalate if urgency is 5 AND (high risk of loss, time-sensitive, or needs carrier claim/account access)
- Otherwise do not escalate

Output JSON only with keys:
- escalate (true/false)
- reason (<= 18 words)
"""

step4_raw = chat(PROMPT_STEP4)
print("\nSTEP 4 OUTPUT (Escalation decision):")
print(step4_raw)

step4 = json.loads(step4_raw)
print("\nSTEP 4 OUTPUT (parsed):")
print(step4)

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}